In [1]:
from pprint import pprint
import sys
sys.path.insert(0, '../src/.')
from utils import check_not_russian_ip

check_not_russian_ip(True)

'Current ip location: NL'

In [2]:
from dotenv import load_dotenv
import os
from pprint import pprint


load_dotenv('../.env')

True

In [3]:
from utils import get_openai_embedding_model


embeddings = get_openai_embedding_model()

In [7]:
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS


files_path = '../pre_installed/books_files'
books_dir = '../pre_installed/books_dir'

def load_book(book_file, book_name, embeddings):
    input_file = os.path.join(files_path, book_file)
    target_dir = os.path.join(books_dir, book_name)

    print(f'* input_file: {input_file}')
    print(f'* target_dir: {target_dir}')

    os.makedirs(target_dir)
    os.system(f'cp "{input_file}" {target_dir}/')

    # Определяем расширение файла
    file_extension = input_file.split('.')[-1].lower()

    # Выбираем загрузчик в зависимости от расширения
    if file_extension == 'txt':
        loader = TextLoader(input_file)
    elif file_extension == 'pdf':
        loader = PyPDFLoader(input_file)
    else:
        raise ValueError(f"Неподдерживаемый формат файла: {file_extension}")
    docs = loader.load()

    splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

    if file_extension == 'pdf':
        splited_doc = []
        for page in docs:
            splited_doc += splitter.create_documents([page.page_content], metadatas=[page.metadata])
    else:
        splited_doc = splitter.create_documents([docs[0].page_content])

    db = FAISS.from_documents(splited_doc, embeddings)  
    db.save_local(f"{target_dir}/faiss_db")

    print(f'* db saved to {target_dir}/faiss_db')




In [ ]:
load_book('The Captains Daughter.txt', 'The_captains_daughter', embeddings)

In [10]:
load_book('George Orwell _1984.pdf', 'George_Orwell_1984', embeddings)

* input_file: ../pre_installed/books_files/George Orwell _1984.pdf
* target_dir: ../pre_installed/books_dir/George_Orwell_1984
* db saved to ../pre_installed/books_dir/George_Orwell_1984/faiss_db


In [5]:
load_book('martin-eden-by-jack-london.pdf', 'Martin_Eden', embeddings)

* input_file: ../pre_installed/books_files/martin-eden-by-jack-london.pdf
* target_dir: ../pre_installed/books_dir/Martin_Eden
* db saved to ../pre_installed/books_dir/Martin_Eden/faiss_db
